In [ ]:
import pathlib
import platform

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import pymc3 as pm

import arviz as ar
import matplotlib.pyplot as pl
from matplotlib import rcParams
import seaborn as sb

### <u>Package Versions<u>

In [ ]:
def pkg_ver(pkgs):
    print('Python & Package Versions')
    print('----------------')
    print(f'PYTHON: {platform.python_version()}')
    for pki in pkgs:
        print(f'{pki.__name__}: {pki.__version__}')
pkg_ver([np, pd, pm, ar, sb])

In [ ]:
ar.style.use('arviz-darkgrid')

### <u>Overview</u>

In this and [a subsequent notebook](), I implement bayesian regression models to predict chlorophyll from satellite and ancillary data. I use a Bayesian framework for all models. For each model, implementation follows the sequence below.

* The model is cast in a Bayesian framework using a probabilistic programming language (PPL);
* A set of prior predictive simulations is conducted to ascertain that model priors are reasonable;
* The model is fit using the No U-Turn Sampler (NUTS) variant of Hamiltonian Monte Carlo, and the data subset from NOMAD 2008;
* Model predictive skill and  uncertainty are quantified via posterior distribution evaluation and posterior predictive simulation.

In a [third notebook](), the skill of the models are compared using Information Criteria (IC) based methods. These include Watanabe Akaike Information Criterion (WAIC) and/or Pareto Smoothed Importance Sampling Leave-One-Out Cross Validation (LOO).

### <u>The Present Notebook's Linear Models</u>
I include 1. models here:
1. A simple maximum-blue band ratio (*MBR*) regression model.
2. An OC4-type \\(4^{th}\\)degree polynomial regression.
3. An OCI-type mixture model where one of either OC4-type or Color Index Model is applied to the data.
$$$$

### <u>Loading the Data</u>
The data was stored previously in a [pandas dataframe](https://pandas.pydata.org). 

In [ ]:
df = pd.read_pickle('./PickleJar/df_main_2_w_CI.pkl')

In [ ]:
df.info()

### <u>Simple Band Ratio</u>
A. Pooled Model.
A pooled regression model is simply a regression on a the whole dataset without partitioning the data into distinct groups (see Partially Pooled Model section next)

In [ ]:
df_small = df[['log_mxBlue2Gr', 'MaxBlueBandIdx', 'log_chlor_a']].dropna()

In [ ]:
X_ = df_small.log_mxBlue2Gr.values
y = df_small.log_chlor_a.values
X_s = (X_ - X_.mean()) / X_.std()
y_s = (y - y.mean()) / y.std()
X_poly = np.c_[X_s, np.power(X_s, 2), np.power(X_s, 3), np.power(X_s, 4)]

In [ ]:
with pm.Model() as mbr_linear_pooled:
    α = pm.Normal('α', mu=0, sd=1)
    β = pm.Normal('β', mu=0, sd=1)
    σ = pm.Exponential('σ', 1)
    μ = α + β * X_s
    likelihood = pm.Normal('likelihood', mu=μ, sd=σ, observed=y_s)

In [ ]:
mlp_graph = pm.model_to_graphviz(mbr_linear_pooled)

B. Hierarchical, Partially Pooled Model

In [ ]:
with pm.Model() as mbr_linear

---
End of this Notebook